In [40]:
import os
import sys
import glob
from os import write

from tqdm import tqdm
import pandas as pd
import numpy as np
import ast

import plotly as py
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

sys.path.append(os.path.realpath(os.path.abspath('..')))
from iDrink import iDrinkUtilities
from iDrink.iDrinkUtilities import get_title_measure_name, get_unit, get_cad, get_setting_axis_name

In [41]:
drive = iDrinkUtilities.get_drivepath()

root_iDrink = os.path.join(drive, 'iDrink')
root_val = os.path.join(root_iDrink, "validation_root")
root_stat = os.path.join(root_val, '04_Statistics')
root_omc = os.path.join(root_val, '03_data', 'OMC_new', 'S15133')
root_data = os.path.join(root_val, "03_data")
root_logs = os.path.join(root_val, "05_logs")

dir_stat_cont = os.path.join(root_stat, '01_continuous')
dir_stat_cat = os.path.join(root_stat, '02_categorical')
dir_results = os.path.join(dir_stat_cont, '01_results')
dir_plots_cont = os.path.join(dir_stat_cont, '02_plots')
dir_plots_cat = os.path.join(dir_stat_cat, '02_plots')

csv_val_trials = os.path.join(root_logs, 'validation_trials.csv')
df_val_trials = pd.read_csv(csv_val_trials, sep=';')

csv_settings = os.path.join(root_logs, 'validation_settings.csv')
df_settings = pd.read_csv(csv_settings, sep=';')

csv_calib_error = os.path.join(root_logs, 'calib_errors.csv')
df_calib_error = pd.read_csv(csv_calib_error, sep=';')

csv_murphy = os.path.join(root_stat, '02_categorical', 'murphy_measures.csv')
df_murphy = pd.read_csv(csv_murphy, sep=';')

csv_failed_trials = os.path.join(root_stat, '04_failed_trials', 'failed_trials.csv')
df_failed_trials = pd.read_csv(csv_failed_trials, sep=';')


csv_cad = os.path.join(root_stat, '02_categorical', 'clinically_acceptable_difference.csv')
df_cad = pd.read_csv(csv_cad, sep=',')

list_identifier = sorted(df_val_trials['identifier'].tolist())

ignore_id_p = ['P11', 'P19']
idx_s_singlecam_full = ['S017', 'S018', 'S019', 'S020', 'S021', 'S022', 'S023', 'S024', 'S025', 'S026']
idx_s_singlecam = ['S017', 'S018']
idx_s_multicam = ['S001', 'S002', 'S003', 'S004', 'S005', 'S006', 'S007', 'S008', 'S009', 'S010', 'S011', 'S012', 'S013', 'S014', 'S015', 'S016']
idx_s_multicam_reduced = ['S001', 'S002']
idx_s_reduced = idx_s_multicam_reduced + idx_s_singlecam
idx_s_full = idx_s_multicam + idx_s_singlecam

In [42]:
reduced_analysis = True # If True, only S001 and S002 from Multicam will be used.
if reduced_analysis:
    file_app = '_reduced'
else:
    file_app = ''
    

## kinematic measures

In [43]:
df_corr_murphy = pd.read_csv(os.path.join(dir_stat_cat, f'stat_murphy_corr{file_app}.csv'), sep=';')
df_corr_murphy['id_s_name'] = df_corr_murphy['id_s'].apply(lambda x: get_setting_axis_name(x))

if reduced_analysis:
    df_corr_murphy = df_corr_murphy[df_corr_murphy['id_s'].isin(idx_s_reduced)]

if 'Unnamed: 0' in df_corr_murphy.columns:
    df_corr_murphy = df_corr_murphy.drop(columns=['Unnamed: 0'])

In [44]:
df_corr_murphy

,id_s,measure,condition,pearson,pearson_p,id_s_name
0,S001,PeakVelocity_mms,affected,0.537724,4.646357e-17,"SimCC, Cams: 1,2,3,4,5"
1,S001,PeakVelocity_mms,unaffected,0.230194,3.139570e-04,"SimCC, Cams: 1,2,3,4,5"
2,S002,PeakVelocity_mms,affected,0.107215,9.270212e-02,"Metrabs, Cams: 1,2,3,4,5"
3,S002,PeakVelocity_mms,unaffected,0.697538,1.864031e-36,"Metrabs, Cams: 1,2,3,4,5"
4,S017,PeakVelocity_mms,affected,0.628377,1.689178e-32,"Single, Cam: 1, filt"
...,...,...,...,...,...,...
291,S002,shoulderAbduction,unaffected,0.790826,7.140105e-53,"Metrabs, Cams: 1,2,3,4,5"
292,S017,shoulderAbduction,affected,0.273235,3.082354e-06,"Single, Cam: 1, filt"
293,S017,shoulderAbduction,unaffected,0.529825,3.036615e-22,"Single, Cam: 1, filt"
294,S018,shoulderAbduction,affected,0.268483,5.850548e-06,"Single, Cam: 1, unfilt"


In [45]:
len_before = len(df_corr_murphy)
df_corr_murphy.dropna(inplace=True)
len_after = len(df_corr_murphy)
print(f'Number of rows dropped: {len_before - len_after}')

Number of rows dropped: 0


In [46]:
df_corr_murphy['measure'].unique()

array(['PeakVelocity_mms', 'elbowVelocity', 'tTopeakV_s',
       'tToFirstpeakV_s', 'tTopeakV_rel', 'tToFirstpeakV_rel',
       'NumberMovementUnits', 'InterjointCoordination',
       'trunkDisplacementMM', 'trunkDisplacementDEG',
       'ShoulderFlexionReaching', 'ElbowExtension', 'shoulderAbduction'],
      dtype=object)

In [47]:
df_grouped = df_corr_murphy.groupby(['id_s']).mean(numeric_only=True)

df_corr_murphy_s = pd.DataFrame(columns = ['id_s', 'id_s_name', 'correlation', 'p_value'])

for index in df_grouped.index:
    id_s = index
    r = df_grouped.loc[index]['pearson']
    p = df_grouped.loc[index]['pearson_p']
    id_s_name = get_setting_axis_name(id_s)

    df_corr_murphy_s = pd.concat([df_corr_murphy_s, pd.DataFrame([[id_s, id_s_name, r, p]], columns = ['id_s', 'id_s_name', 'correlation', 'p_value'], index = [0])], ignore_index=True)

In [48]:
df_grouped = df_corr_murphy.groupby(['id_s', 'measure']).mean(numeric_only=True)

df_plot = pd.DataFrame(columns = ['id_s', 'id_s_name', 'measure', 'correlation', 'p_value'])

for index in df_grouped.index:
    id_s = index[0]
    measure = index[1]
    r = df_grouped.loc[index]['pearson']
    p = df_grouped.loc[index]['pearson_p']
    id_s_name = get_setting_axis_name(id_s)
    
    df_plot = pd.concat([df_plot, pd.DataFrame([[id_s, id_s_name, measure, r, p]], columns = ['id_s', 'id_s_name', 'measure', 'correlation', 'p_value'], index = [0])], ignore_index=True)

In [49]:
df_plot

,id_s,id_s_name,measure,correlation,p_value
0,S001,"SimCC, Cams: 1,2,3,4,5",ElbowExtension,0.889864,1.501644e-66
1,S001,"SimCC, Cams: 1,2,3,4,5",InterjointCoordination,0.936175,8.583237e-93
2,S001,"SimCC, Cams: 1,2,3,4,5",NumberMovementUnits,0.385096,2.737456e-09
3,S001,"SimCC, Cams: 1,2,3,4,5",PeakVelocity_mms,0.383959,1.569785e-04
4,S001,"SimCC, Cams: 1,2,3,4,5",ShoulderFlexionReaching,0.773725,6.344413e-31
5,S001,"SimCC, Cams: 1,2,3,4,5",elbowVelocity,0.703622,2.061824e-30
6,S001,"SimCC, Cams: 1,2,3,4,5",shoulderAbduction,0.703301,8.201904e-35
7,S001,"SimCC, Cams: 1,2,3,4,5",tToFirstpeakV_rel,0.487592,3.379935e-08
8,S001,"SimCC, Cams: 1,2,3,4,5",tToFirstpeakV_s,0.624736,7.051257e-17
9,S001,"SimCC, Cams: 1,2,3,4,5",tTopeakV_rel,0.487592,3.379935e-08


In [50]:
df_plot['id_s'].unique() 

array(['S001', 'S002', 'S017', 'S018'], dtype=object)

In [51]:


fig = px.bar(df_plot, x='id_s_name', y='correlation', color='measure', title=f'<b>Pearson Correlation<b>', labels={'correlation': 'Correlation [r] Correlation', 'Settings': 'Subject ID', 'measure': 'Kin. Measure'},
             template = 'plotly', barmode="group")

fig.add_hline(y=0.9, line_dash='dot', line_color='red', name = 'minimum correlation')
#fig.add_hline(y=-0.9, line_dash='dot', line_color='red', name = 'minimum correlation')

fig.update_layout(title= f'<b>Pearson Correlation of kinematic measures and settings<b>',
          xaxis_title=f'Settings',
          yaxis_title=f'Correlation [r]',
                  width=1300, height=600
          )

fig.show()

os.makedirs(os.path.join(dir_plots_cat, '04_correlation'), exist_ok=True)

path = os.path.join(dir_plots_cat, '04_correlation', f'pearson_corr_all.html')
fig.write_html(path)

path = os.path.join(dir_plots_cat, '04_correlation', f'pearson_corr_all.png')
fig.write_image(path, scale = 5)

In [52]:
fig = px.bar(df_plot[df_plot['id_s'].isin(idx_s_reduced)], x='id_s_name', y='correlation', color='measure', title=f'Pearson Correlation', labels={'correlation': 'Pearson Correlation', 'id_s': 'Subject ID', 'measure': 'Kin. Measure'},
             template = 'plotly', barmode="group")

fig.add_hline(y=0.9, line_dash='dot', line_color='red', name = 'minimum correlation')

fig.update_layout(title= f'Pearson Correlation of chosen settings',
          xaxis_title=f'Settings',
          yaxis_title=f'Correlation [r]',
                  width=1300, height=600
          )

fig.show()

os.makedirs(os.path.join(dir_plots_cat, '04_correlation'), exist_ok=True)

path = os.path.join(dir_plots_cat, '04_correlation', f'pearson_corr_reduced.html')
fig.write_html(path)

path = os.path.join(dir_plots_cat, '04_correlation', f'pearson_corr_reduced.png')
fig.write_image(path, scale = 5)

In [53]:
measures = df_plot['measure'].unique()

for measure in measures:
    df_plot_measure = df_plot[df_plot['measure'] == measure]
    title_name = get_title_measure_name(measure)
    
    fig = px.bar(df_plot_measure[df_plot_measure['id_s'].isin(idx_s_reduced)], x='id_s_name', y='correlation', title=f'Pearson Correlation of {title_name}', labels={'correlation': 'Pearson Correlation', 'id_s': 'Subject ID', 'measure': 'Kin. Measure'},
                 template = 'plotly')
    
    fig.add_hline(y=0.9, line_dash='dot', line_color='red', name = 'minimum correlation')
    
    fig.update_layout(title= f'Pearson Correlation of {title_name}',
              xaxis_title=f'Settings',
              yaxis_title=f'Correlation [r]',
                      width=1300, height=600
              )
    
    fig.show()
    os.makedirs(os.path.join(dir_plots_cat, '04_correlation'), exist_ok=True)
    
    path = os.path.join(dir_plots_cat, '04_correlation', f'pearson_corr_{measure}.html')
    fig.write_html(path)
    
    path = os.path.join(dir_plots_cat, '04_correlation', f'pearson_corr_{measure}.png')
    fig.write_image(path, scale = 5)

## Timeseries


In [54]:
csv_corr_ts = os.path.join(dir_stat_cont, '01_results', 'time_series_correlation.csv')
df_corr_ts = pd.read_csv(csv_corr_ts, sep=';')

metrics_ts = ['hand_vel', 'elbow_vel', 'trunk_disp', 'elbow_flex_pos', 'shoulder_flex_pos', 'shoulder_abduction_pos']

df_corr_ts = df_corr_ts[(df_corr_ts['dynamic'] == 'fixed') & (df_corr_ts['metric'].isin(metrics_ts)) & (~df_corr_ts['id_p'].isin(ignore_id_p))]

In [55]:
df_corr_ts_s = pd.DataFrame(columns=['id_s', 'id_s_name', 'correlation', 'p_value'])
df_grouped = df_corr_ts[df_corr_ts['id_s'].isin(idx_s_full)].groupby(['id_s']).mean(numeric_only=True)

for index in df_grouped.index:
    id_s = index
    r = df_grouped.loc[index]['pearson']
    p = df_grouped.loc[index]['pearson_p']
    id_s_name = get_setting_axis_name(id_s)

    df_corr_ts_s = pd.concat([df_corr_ts_s, pd.DataFrame([[id_s, id_s_name, r, p]],
                                                    columns=['id_s', 'id_s_name', 'correlation', 'p_value'],
                                                    index=[0])], ignore_index=True)
df_corr_ts_s.sort_values(by='id_s', inplace=True)

### All Settings Joint Measures and Time-series

In [56]:
merged = pd.concat([df_corr_murphy_s, df_corr_ts_s])

df_corr_murphy_and_ts = merged.groupby(['id_s', 'id_s_name'], as_index=False).mean(numeric_only=True)

In [57]:
df_corr_murphy_and_ts.sort_values(by='correlation', ascending=False)

,id_s,id_s_name,correlation,p_value
1,S002,"Metrabs, Cams: 1,2,3,4,5",0.767733,0.004444
0,S001,"SimCC, Cams: 1,2,3,4,5",0.717349,0.010262
16,S017,"Single, Cam: 1, filt",0.688211,0.003339
7,S008,"Metrabs, Cams: 2,3,4",0.670515,0.040703
17,S018,"Single, Cam: 1, unfilt",0.634916,0.004246
12,S013,"SimCC, Cams: 2,4",0.496257,0.042706
13,S014,"Metrabs, Cams: 2,4",0.474347,0.078661
5,S006,"Metrabs, Cams: 1,3,5",0.388474,0.053147
6,S007,"SimCC, Cams: 2,3,4",0.380477,0.072564
4,S005,"SimCC, Cams: 1,3,5",0.261180,0.066029


In [58]:
fig = px.bar(df_corr_murphy_and_ts.sort_values(by='correlation', ascending=False), x='id_s_name', y='correlation', template = 'plotly')

#fig.add_hline(y=0.9, line_dash='dot', line_color='red', name = 'corr = 0.9')

fig.update_layout(title= f'Pearson Correlation for all settings',
                  yaxis_title=f'Mean Correlation of Time-Series and Measures [r]',
                    width=1300, height=600
            )

fig.show()

os.makedirs(os.path.join(dir_plots_cont, '04_correlation'), exist_ok=True)

path = os.path.join(dir_plots_cont, '04_correlation', f'pearson_corr_joint_mean.html')
fig.write_html(path)

path = os.path.join(dir_plots_cont, '04_correlation', f'pearson_corr_joint_mean.png')
fig.write_image(path, scale = 5)

### Multi-Cam

Let's first look at all Multi-cam Settings

In [59]:
df_plot = pd.DataFrame(columns = ['id_s', 'id_s_name', 'id_p', 'ml', 'correlation', 'p_value'])
df_grouped = df_corr_ts[df_corr_ts['dynamic'] == 'fixed'].groupby(['id_s', 'id_p']).mean(numeric_only=True)

for index in df_grouped.index:        
    id_s = index[0]
    id_p = index[1]
    r = df_grouped.loc[index]['pearson']
    p = df_grouped.loc[index]['pearson_p']
    id_s_name = get_setting_axis_name(id_s)
    ml = id_s_name.split(',')[0]
    
    df_plot = pd.concat([df_plot, pd.DataFrame([[id_s, id_s_name, id_p, ml,  r, p]], columns = ['id_s', 'id_s_name', 'id_p', 'ml', 'correlation', 'p_value'], index = [0])], ignore_index=True)

df_plot = df_plot[df_plot['id_s'].isin(idx_s_multicam)]

In [60]:
fig = px.bar(df_plot.sort_values(by='id_s'), x='id_s_name', y='correlation', color='id_p', template = 'plotly', barmode='group', labels={"id_p": "Patient"})

fig.add_hline(y=0.9, line_dash='dot', line_color='red', name = 'corr = 0.9')

fig.update_layout(title= f'Pearson Correlation for multi-cam settings',
          xaxis_title=f'Setting',
          yaxis_title=f'Correlation [r]',
                  width=1300, height=600
          )

os.makedirs(os.path.join(dir_plots_cont, '04_correlation'), exist_ok=True)

path = os.path.join(dir_plots_cont, '04_correlation', f'pearson_corr_multi_cam.html')
fig.write_html(path)

path = os.path.join(dir_plots_cont, '04_correlation', f'pearson_corr_multi_cam.png')
fig.write_image(path, scale = 5)

In [61]:

df_grouped = df_plot.groupby(['id_p', 'ml']).mean(numeric_only=True)
df_plot = pd.DataFrame(columns = ['id_p', 'ml', 'correlation', 'p_value'])

for index in df_grouped.index:        
    id_p = index[0]
    ml = index[1]
    r = df_grouped.loc[index]['correlation']
    p = df_grouped.loc[index]['p_value']

    
    df_plot = pd.concat([df_plot, pd.DataFrame([[id_p, ml,  r, p]], columns = ['id_p', 'ml', 'correlation', 'p_value'], index = [0])], ignore_index=True)

In [62]:
fig = px.bar(df_plot, x= 'ml', y='correlation', color='id_p', template = 'plotly', barmode='group', labels={"id_p": "Patient", "ml": "Pose Estimation Model", 'correlation': 'Correlation [r]'})

fig.add_hline(y=0.9, line_dash='dot', line_color='red', name = 'corr = 0.9')

fig.update_layout(title= f'Pearson Correlation for multi-cam settings',
                  width=1300, height=600
          )

os.makedirs(os.path.join(dir_plots_cont, '04_correlation'), exist_ok=True)

path = os.path.join(dir_plots_cont, '04_correlation', f'pearson_corr_multi_cam_model.html')
fig.write_html(path)

path = os.path.join(dir_plots_cont, '04_correlation', f'pearson_corr_multi_cam_model.png')
fig.write_image(path, scale = 5)


Settings using less than five cameras show bad performance.
### SingleCam

In [63]:
df_plot = pd.DataFrame(columns = ['id_s', 'id_s_name', 'id_p', 'correlation', 'p_value'])
df_grouped = df_corr_ts[(df_corr_ts['dynamic'] == 'fixed') & (df_corr_ts['id_s'].isin(idx_s_singlecam))].groupby(['id_s', 'id_p']).mean(numeric_only=True)

for index in df_grouped.index:
    id_s = index[0]
    id_p = index[1]
    r = df_grouped.loc[index]['pearson']
    p = df_grouped.loc[index]['pearson_p']
    id_s_name = get_setting_axis_name(id_s)
    
    df_plot = pd.concat([df_plot, pd.DataFrame([[id_s, id_s_name, id_p, r, p]], columns = ['id_s', 'id_s_name', 'id_p', 'correlation', 'p_value'], index = [0])], ignore_index=True)
    df_plot.sort_values(by='id_s', inplace=True)

In [64]:
fig = px.bar(df_plot, x='id_s', y='correlation', color='id_p', template = 'plotly', barmode='group', labels={"id_p": "Patient"})

fig.add_hline(y=0.9, line_dash='dot', line_color='red', name = 'corr = 0.9')

fig.update_layout(title= f'Pearson Correlation for single-cam settings',
                  xaxis_title=f'Setting',
                  yaxis_title=f'Correlation [r]',
                  width=1300, height=600
                  )

path = os.path.join(dir_plots_cont, '04_correlation', 'pearson_corr_single_cam.html')
fig.write_html(path)

path = os.path.join(dir_plots_cont, '04_correlation', 'pearson_corr_single_cam.png')
fig.write_image(path)


### All Settings

In [65]:
idx_s_to_use = idx_s_reduced if reduced_analysis else idx_s_full
idx_p_to_use = [p for p in df_corr_ts['id_p'].unique() if p not in ignore_id_p]

In [66]:
df_plot = pd.DataFrame(columns = ['id_s', 'id_s_name', 'id_p', 'correlation', 'p_value'])
df_grouped = df_corr_ts[(df_corr_ts['dynamic'] == 'fixed') & (df_corr_ts['id_s'].isin(idx_s_to_use)) & (df_corr_ts['id_p'].isin(idx_p_to_use))].groupby(['id_s', 'id_p']).mean(numeric_only=True)

for index in df_grouped.index:
    id_s = index[0]
    id_p = index[1]
    r = df_grouped.loc[index]['pearson']
    p = df_grouped.loc[index]['pearson_p']
    id_s_name = get_setting_axis_name(id_s)
    
    df_plot = pd.concat([df_plot, pd.DataFrame([[id_s, id_s_name, id_p, r, p]], columns = ['id_s', 'id_s_name',  'id_p', 'correlation', 'p_value'], index = [0])], ignore_index=True)
    df_plot.sort_values(by='id_s', inplace=True)

In [67]:
fig = px.bar(df_plot.sort_values(by='id_s'), x='id_s_name', y='correlation', color='id_p', template = 'plotly', barmode='group', labels={"id_p": "Patient"})

fig.add_hline(y=0.9, line_dash='dot', line_color='red', name = 'corr = 0.9')

fig.update_layout(title= f'Mean Pearson Correlation for settings and patients',
                  xaxis_title=f'Setting',
                  yaxis_title=f'Correlation [r]',
                  width=1300, height=600
          )

path = os.path.join(dir_plots_cont, '04_correlation', f'pearson_corr_all_settings{file_app}.html')
fig.write_html(path)

path = os.path.join(dir_plots_cont, '04_correlation', f'pearson_corr_all_settings{file_app}.png')
fig.write_image(path, scale = 5)


In [68]:
df_corr_ts

,id_s,id_p,id_t,condition,side,dynamic,metric,pearson,pearson_p
0,S001,P07,T002,unaffected,L,fixed,hand_vel,0.991326,9.745339e-302
1,S001,P07,T002,unaffected,L,fixed,elbow_vel,0.982080,1.661938e-248
2,S001,P07,T002,unaffected,L,fixed,trunk_disp,0.992713,1.502667e-314
4,S001,P07,T002,unaffected,L,fixed,elbow_flex_pos,0.995176,0.000000e+00
5,S001,P07,T002,unaffected,L,fixed,shoulder_flex_pos,0.999248,0.000000e+00
...,...,...,...,...,...,...,...,...,...
63386,S026,P15,T090,affected,L,fixed,elbow_vel,0.926834,1.670835e-179
63387,S026,P15,T090,affected,L,fixed,trunk_disp,0.960560,6.671215e-234
63389,S026,P15,T090,affected,L,fixed,elbow_flex_pos,0.983898,5.795650e-314
63390,S026,P15,T090,affected,L,fixed,shoulder_flex_pos,0.948707,1.185178e-210


In [69]:
df_plot = pd.DataFrame(columns = ['id_s', 'id_s_name', 'id_p', 'correlation', 'p_value'])
df_grouped = df_corr_ts[(df_corr_ts['dynamic'] == 'fixed') & (df_corr_ts['id_s'].isin(idx_s_to_use)) & (df_corr_ts['id_p'].isin(idx_p_to_use))].groupby(['id_s', 'metric']).mean(numeric_only=True)

for index in df_grouped.index:
    id_s = index[0]
    metric = index[1]
    r = df_grouped.loc[index]['pearson']
    p = df_grouped.loc[index]['pearson_p']
    id_s_name = get_setting_axis_name(id_s)
    
    df_plot = pd.concat([df_plot, pd.DataFrame([[id_s, id_s_name, metric, r, p]], columns = ['id_s', 'id_s_name', 'metric', 'correlation', 'p_value'], index = [0])], ignore_index=True)
    df_plot.sort_values(by='id_s', inplace=True)

In [70]:
fig = px.bar(df_plot.sort_values(by='id_s'), x='id_s_name', y='correlation', color='metric', template = 'plotly', barmode='group', labels={"metric": "Kin. metrics"})

fig.add_hline(y=0.9, line_dash='dot', line_color='red', name = 'corr = 0.9')

fig.update_layout(title= f'Pearson Correlation for chosen settings and kinematic metrics',
                  yaxis_title=f'Mean Correlation [r]',
                  xaxis_title='Settings',
                  width=1300, height=600
                  )

path = os.path.join(dir_plots_cont, '04_correlation', f'pearson_corr_kinematics{file_app}.html')
fig.write_html(path)

path = os.path.join(dir_plots_cont, '04_correlation', f'pearson_corr_kinematics{file_app}.png')
fig.write_image(path, scale = 5)

In [71]:
fig.show()